# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
# X['home_ownership'].unique()

In [6]:
clean_df = df.copy()

clean_df['home_ownership'] = clean_df['home_ownership'].apply(lambda x: 1 if x == 'RENT' else 0)

clean_df['loan_status'] = clean_df['loan_status'].apply(lambda x: 0 if x == 'low_risk' else 1)

clean_df['application_type'] = clean_df['application_type'].apply(lambda x: 0 if x == 'Joint App' else 1)

clean_df['verification_status'] = clean_df['verification_status'] \
                                    .apply(lambda x: {'Not Verified': 0, 'Verified': 1, 'Source Verified': 2}[x])

months = {'Jan-2019': 1, 'Feb-2019': 2, 'Mar-2019': 3, 'Apr-2019': 4, 'May-2019': 5, 'Jun-2019': 6,
          'Jul-2019': 7, 'Aug-2019': 8, 'Sep-2019': 9, 'Oct-2019': 10, 'Nov-2019': 11,
          'Dec-2019': 12}

for k, v in months.items():
    clean_df.replace(to_replace=k, value=v, inplace=True)

    
clean_df.replace(to_replace='n', value=0, inplace=True)
clean_df.replace(to_replace='y', value=1, inplace=True)
clean_df.replace(to_replace='N', value=0, inplace=True)
clean_df.replace(to_replace='Y', value=1, inplace=True)
clean_df.replace(to_replace='w', value=1, inplace=True)
clean_df.replace(to_replace='f', value=1, inplace=True)
clean_df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,1,66000.0,2,3,0,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,0,105000.0,1,3,0,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,0,56000.0,1,3,0,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,1,92000.0,1,3,0,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,0,52000.0,0,3,0,0,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


# Split the Data into Training and Testing

In [7]:
# Create our features
X = clean_df[columns]

# X['home_ownership'] = X['home_ownership'].apply(lambda x:{'RENT':0, 'MORTGAGE':1, 'OWN':2, 'ANY':3}[x])

# Create our target
y = df[target]

In [8]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,0.357659,8.821371e+04,0.895985,1.726172,0.005042,0.0,21.778153,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.479314,1.155800e+05,0.917165,0.743862,0.070831,0.0,20.199244,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,1.000000,0.000000,0.0,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,0.000000,5.000000e+04,0.000000,1.000000,0.000000,0.0,13.890000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,0.000000,7.300000e+04,1.000000,2.000000,0.000000,0.0,19.760000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,1.000000,1.040000e+05,2.000000,2.000000,0.000000,0.0,26.660000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,1.000000,8.797500e+06,2.000000,3.000000,1.000000,0.0,999.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [9]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [11]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_test)

Counter({'loan_status': 1})

In [12]:
X_resampled

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,1600.0,0.0881,50.74,0,35964.0,0,3,0,0,24.12,...,100.0,0.0,0.0,0.0,148200.0,14292.0,58300.0,15000.0,0,0
1,9000.0,0.0756,280.21,0,41000.0,0,2,0,0,16.89,...,95.0,33.3,0.0,0.0,68292.0,52587.0,24500.0,40492.0,0,0
2,10000.0,0.1033,214.10,0,112000.0,1,2,0,0,17.75,...,92.6,20.0,0.0,0.0,416023.0,78944.0,65400.0,80642.0,0,0
3,36000.0,0.1033,1167.21,1,120000.0,0,3,0,0,19.95,...,100.0,40.0,0.0,0.0,116200.0,73826.0,85200.0,30000.0,0,0
4,18000.0,0.0881,570.81,0,51000.0,1,1,0,0,19.11,...,95.5,25.0,0.0,0.0,221075.0,126195.0,36500.0,96715.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102727,20000.0,0.1102,435.05,0,117000.0,0,3,1,0,11.72,...,97.9,0.0,0.0,0.0,427774.0,15502.0,71700.0,18720.0,0,0
102728,10000.0,0.1356,339.65,0,101000.0,2,1,1,0,19.13,...,86.7,0.0,0.0,0.0,192300.0,64513.0,20800.0,64800.0,0,0
102729,21000.0,0.0702,648.62,1,70000.0,2,1,1,0,16.42,...,100.0,0.0,0.0,0.0,129042.0,42302.0,72300.0,52742.0,0,0
102730,20000.0,0.1502,693.51,1,36000.0,1,2,1,0,28.57,...,100.0,50.0,0.0,0.0,55000.0,28207.0,8000.0,18500.0,0,0


In [13]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [15]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

predictions = model.predict(X_test)
acc_score = balanced_accuracy_score(y_test, predictions)
acc_score

0.6635657573007067

In [16]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, predictions)
cm

array([[   70,    31],
       [ 6259, 10845]], dtype=int64)

In [17]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.69      0.63      0.02      0.66      0.44       101
   low_risk       1.00      0.63      0.69      0.78      0.66      0.44     17104

avg / total       0.99      0.63      0.69      0.77      0.66      0.44     17205



### SMOTE Oversampling

In [18]:
# Resample the training data with SMOTE
Counter(y_resampled)

Counter({'loan_status': 1})

In [19]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [21]:
# Calculated the balanced accuracy score
predictions = model.predict(X_test)
balanced_accuracy_score(y_test, predictions)

0.6635657573007067

In [22]:
# Display the confusion matrix
confusion_matrix(y_test, predictions)

array([[   70,    31],
       [ 6259, 10845]], dtype=int64)

In [23]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.69      0.63      0.02      0.66      0.44       101
   low_risk       1.00      0.63      0.69      0.78      0.66      0.44     17104

avg / total       0.99      0.63      0.69      0.77      0.66      0.44     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [24]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
Counter(y_resampled)

Counter({'loan_status': 1})

In [25]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [27]:
# Calculated the balanced accuracy score
predictions = model.predict(X_test)
balanced_accuracy_score(y_test, predictions)

0.6635657573007067

In [28]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[   70,    31],
       [ 6259, 10845]], dtype=int64)

In [29]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.69      0.63      0.02      0.66      0.44       101
   low_risk       1.00      0.63      0.69      0.78      0.66      0.44     17104

avg / total       0.99      0.63      0.69      0.77      0.66      0.44     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [30]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
Counter(y_resampled)

Counter({'loan_status': 1})

In [31]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [32]:
# Calculated the balanced accuracy score
predictions = model.predict(X_test)
balanced_accuracy_score(y_test, predictions)

0.6635657573007067

In [33]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[   70,    31],
       [ 6259, 10845]], dtype=int64)

In [35]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.69      0.63      0.02      0.66      0.44       101
   low_risk       1.00      0.63      0.69      0.78      0.66      0.44     17104

avg / total       0.99      0.63      0.69      0.77      0.66      0.44     17205

